# **Final Project Task 1 - Census Data Preprocess**

Requirements

- Target variable specification:
    - The target variable for this project is hours-per-week. 
    - Ensure all preprocessing steps are designed to support regression analysis on this target variable.
- Encode data  **3p**
- Handle missing values if any **1p**
- Correct errors, inconsistencies, remove duplicates if any **1p**
- Outlier detection and treatment if any **1p**
- Normalization / Standardization if necesarry **1p**
- Feature engineering **3p**
- Train test split, save it.
- Others?


Deliverable:

- Notebook code with no errors.
- Preprocessed data as csv.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [2]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

data = pd.read_csv(data_url, header=None, names=columns, na_values=" ?", skipinitialspace=True)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# Eliminare duplicate
data = data.drop_duplicates()

# Verificare valori lipsă
data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

drop_duplicates() elimină rândurile identice, pentru a evita bias-ul în model.
isna().sum() identifică numărul de valori lipsă pentru fiecare coloană.

In [4]:
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns

Q1 = data[numeric_cols].quantile(0.25)
Q3 = data[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

outliers = ((data[numeric_cols] < (Q1 - 1.5 * IQR)) | 
            (data[numeric_cols] > (Q3 + 1.5 * IQR)))

outlier_counts = outliers.sum()
outlier_counts

age                142
fnlwgt             993
education-num     1193
capital-gain      2712
capital-loss      1519
hours-per-week    9002
dtype: int64

Se observa multipli outlieri în setul de date

In [5]:
numeric_cols = data.select_dtypes(include=["int64", "float64"]).columns

def cap_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower, lower,
                        np.where(df[col] > upper, upper, df[col]))

for col in numeric_cols:
    cap_outliers(data, col)

Folosim metoda IQR (Interquartile Range) pentru identificarea și limitarea outlierilor.

Valorile extreme sunt capped la limita superioară și inferioară, astfel încât să nu afecteze antrenarea modelului.

In [6]:
# education-num este echivalent numeric al education
data = data.drop(columns=["education"])

In [7]:
# Convertirea variabilei într-o variabilă binară
data["income_binary"] = data["income"].map({"<=50K": 0, ">50K": 1})
data = data.drop(columns=["income"])

In [8]:
# Aplicarea transformării logaritmice pentru capital-gain și capital-loss
# Pentru a reduce asimetria distribuției
data["capital-gain"] = np.log1p(data["capital-gain"])
data["capital-loss"] = np.log1p(data["capital-loss"])

education-num este deja numeric, deci coloana education textuală este eliminată.

income este convertită într-o variabilă binară pentru eventuale modele viitoare.

np.log1p aplicata pe capital-gain și capital-loss reduce asimetria extremă a distribuției și stabilizează varianța.

In [9]:
# Definirea targetului și a caracteristicilor
target = "hours-per-week"

X = data.drop(columns=[target])
y = data[target]

X conține toate variabilele explicative.
y este variabila țintă hours-per-week pentru regresie.

In [10]:
# Prepocesarea datelor
# Definirea transformărilor pentru variabile numerice și categorice
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object", "string"]).columns

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

Se definesc pipeline-uri separate pentru variabile numerice și categorice.

Numerice: înlocuire mediană + standardizare (scalare la medie 0 și varianță 1).

Categorice: înlocuire modă + codificare One-Hot.

ColumnTransformer procesează numeric și categoric în același timp.

In [12]:
# Train -test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

80% date pentru antrenare, 20% pentru test.

random_state fixează generatorul aleator pentru reproducibilitate.

In [ ]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

fit_transform aplicat pe train calculează parametrii (mediană, medie, encoding) și transformă datele.

transform pe test aplică aceiași parametri calculați pe train, evitând leakage-ul.

In [ ]:
# Nume coloane după One-Hot Encoding
encoded_cat_cols = preprocessor.named_transformers_["cat"] \
    .named_steps["encoder"] \
    .get_feature_names_out(categorical_features)

final_columns = list(numeric_features) + list(encoded_cat_cols)

X_train_df = pd.DataFrame(X_train_processed, columns=final_columns)
X_test_df = pd.DataFrame(X_test_processed, columns=final_columns)

train_df = pd.concat([X_train_df, y_train.reset_index(drop=True)], axis=1)
test_df = pd.concat([X_test_df, y_test.reset_index(drop=True)], axis=1)

train_df.to_csv("train_preprocessed.csv", index=False)
test_df.to_csv("test_preprocessed.csv", index=False)

Se obțin numele coloanelor după One-Hot encoding.

Se reconstruiește un DataFrame pentru a păstra structura clară.

Se adaugă coloana target (hours-per-week) la datasetul final.

Se salvează datele preprocesate în fișiere CSV simple și descriptive.